In [1]:
# Import Data Preprocessing and Wrangling libraries
import re
from tqdm.notebook import tqdm
import pandas as pd 
import numpy as np
from datetime import datetime
import dateutil.parser

# Import NLP Libraries
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

# Import Visualization Libraries

import matplotlib.pyplot as plt
from nltk.corpus import stopwords

In [2]:
tweets_df = pd.read_csv("musk_quote_tweets.csv")
tweets_df.head()

,orig_tweet_id,orig_tweet_created_at,orig_tweet_text,orig_tweet_url,orig_tweet_twitter_url,orig_tweet_username,orig_tweet_retweet_count,orig_tweet_reply_count,orig_tweet_like_count,orig_tweet_quote_count,...,orig_tweet_bookmark_count,musk_tweet_id,musk_quote_tweet,musk_quote_retweet_count,musk_quote_reply_count,musk_quote_like_count,musk_quote_quote_count,musk_quote_view_count,musk_quote_bookmark_count,musk_quote_created_at
0,1655977349530243074,2023-05-09 16:45:41+00:00,Hot take:\n\n@Twitter can become the world's n...,https://x.com/GuyDealership/status/16559773495...,https://twitter.com/GuyDealership/status/16559...,GuyDealership,632,497,7730,123,...,387,1655978502187778073,Yup,3255,3747,39533,225,11392206.0,281,2023-05-09 16:50:16+00:00
1,1655968201422012418,2023-05-09 16:09:20+00:00,If Legacy Media is going to shove bigotry porn...,https://x.com/TheRabbitHole84/status/165596820...,https://twitter.com/TheRabbitHole84/status/165...,TheRabbitHole84,1409,412,5328,264,...,633,1655968899903418373,Massive public manipulation,9811,2694,49528,534,14404853.0,1241,2023-05-09 16:12:06+00:00
2,1647327385342320640,2023-04-15 19:53:49+00:00,Starlink provides internet to the most remote ...,https://x.com/teslaownersSV/status/16473273853...,https://twitter.com/teslaownersSV/status/16473...,teslaownersSV,1253,800,8472,175,...,173,1647339741610926080,💯,8257,4793,106036,376,18520248.0,366,2023-04-15 20:42:55+00:00
3,1646226917387796491,2023-04-12 19:00:57+00:00,National Public Radio Denies Being National Or...,https://x.com/TheBabylonBee/status/16462269173...,https://twitter.com/TheBabylonBee/status/16462...,TheBabylonBee,4781,701,37752,442,...,167,1646228474628280326,🤣🤣,10198,5076,108462,430,16690340.0,340,2023-04-12 19:07:08+00:00
4,1640016339011076097,2023-03-26 15:42:19+00:00,Arthur C. Clarke about the future of AI. \n—21...,https://x.com/Rainmaker1973/status/16400163390...,https://twitter.com/Rainmaker1973/status/16400...,Rainmaker1973,5409,737,20773,814,...,3333,1640171198091866114,Prescient,9193,5118,56272,572,25169601.0,3792,2023-03-27 01:57:41+00:00


In [3]:
tweets_df.drop(['orig_tweet_url', 'orig_tweet_twitter_url', 'orig_tweet_quote_count'], axis=1, inplace=True)
tweets_df.head()

,orig_tweet_id,orig_tweet_created_at,orig_tweet_text,orig_tweet_username,orig_tweet_retweet_count,orig_tweet_reply_count,orig_tweet_like_count,orig_tweet_view_count,orig_tweet_bookmark_count,musk_tweet_id,musk_quote_tweet,musk_quote_retweet_count,musk_quote_reply_count,musk_quote_like_count,musk_quote_quote_count,musk_quote_view_count,musk_quote_bookmark_count,musk_quote_created_at
0,1655977349530243074,2023-05-09 16:45:41+00:00,Hot take:\n\n@Twitter can become the world's n...,GuyDealership,632,497,7730,12218484.0,387,1655978502187778073,Yup,3255,3747,39533,225,11392206.0,281,2023-05-09 16:50:16+00:00
1,1655968201422012418,2023-05-09 16:09:20+00:00,If Legacy Media is going to shove bigotry porn...,TheRabbitHole84,1409,412,5328,14775379.0,633,1655968899903418373,Massive public manipulation,9811,2694,49528,534,14404853.0,1241,2023-05-09 16:12:06+00:00
2,1647327385342320640,2023-04-15 19:53:49+00:00,Starlink provides internet to the most remote ...,teslaownersSV,1253,800,8472,20877634.0,173,1647339741610926080,💯,8257,4793,106036,376,18520248.0,366,2023-04-15 20:42:55+00:00
3,1646226917387796491,2023-04-12 19:00:57+00:00,National Public Radio Denies Being National Or...,TheBabylonBee,4781,701,37752,18478251.0,167,1646228474628280326,🤣🤣,10198,5076,108462,430,16690340.0,340,2023-04-12 19:07:08+00:00
4,1640016339011076097,2023-03-26 15:42:19+00:00,Arthur C. Clarke about the future of AI. \n—21...,Rainmaker1973,5409,737,20773,27514757.0,3333,1640171198091866114,Prescient,9193,5118,56272,572,25169601.0,3792,2023-03-27 01:57:41+00:00


In [4]:
elon_tweets_df = tweets_df.loc[tweets_df['orig_tweet_username'] == "elonmusk"]
elon_tweets_df.head()

,orig_tweet_id,orig_tweet_created_at,orig_tweet_text,orig_tweet_username,orig_tweet_retweet_count,orig_tweet_reply_count,orig_tweet_like_count,orig_tweet_view_count,orig_tweet_bookmark_count,musk_tweet_id,musk_quote_tweet,musk_quote_retweet_count,musk_quote_reply_count,musk_quote_like_count,musk_quote_quote_count,musk_quote_view_count,musk_quote_bookmark_count,musk_quote_created_at
14,1668843112955891712,2023-06-14 04:49:38+00:00,Bold,elonmusk,49375,18263,334997,127858792.0,6344,1668868113725550592,"Once again, I’d like to offer this platform to...",18655,21155,208132,1974,59238311.0,996,2023-06-14 06:28:59+00:00
21,1665784889008873476,2023-06-05 18:17:20+00:00,https://t.co/OT6bqzUK8P,elonmusk,10205,7183,58206,34809220.0,2905,1665788623797600258,On now,5473,5188,51793,295,18446361.0,423,2023-06-05 18:32:11+00:00
40,1656748197308674048,2023-05-11 19:48:45+00:00,Excited to announce that I’ve hired a new CEO ...,elonmusk,30150,31364,361878,120525183.0,2900,1657050349608501249,I am excited to welcome Linda Yaccarino as the...,24273,48138,213386,9439,84042223.0,2782,2023-05-12 15:49:24+00:00
53,999367582271422464,2018-05-23 19:12:53+00:00,Going to create a site where the public can ra...,elonmusk,41664,17931,235900,NaN,1333,1650637588078665728,Or maybe just X,5792,5999,63709,504,17064005.0,355,2023-04-24 23:07:23+00:00
75,1619415871902056449,2023-01-28 19:23:25+00:00,"@atensnut Nonetheless, I apologize",elonmusk,414,1166,8046,1923932.0,59,1619504327663714304,@SRuhle https://t.co/79KLYgGGLj,123,333,3793,35,1359014.0,16,2023-01-29 01:14:55+00:00


In [5]:
quote_df = tweets_df.copy()
quote_df.head()

,orig_tweet_id,orig_tweet_created_at,orig_tweet_text,orig_tweet_username,orig_tweet_retweet_count,orig_tweet_reply_count,orig_tweet_like_count,orig_tweet_view_count,orig_tweet_bookmark_count,musk_tweet_id,musk_quote_tweet,musk_quote_retweet_count,musk_quote_reply_count,musk_quote_like_count,musk_quote_quote_count,musk_quote_view_count,musk_quote_bookmark_count,musk_quote_created_at
0,1655977349530243074,2023-05-09 16:45:41+00:00,Hot take:\n\n@Twitter can become the world's n...,GuyDealership,632,497,7730,12218484.0,387,1655978502187778073,Yup,3255,3747,39533,225,11392206.0,281,2023-05-09 16:50:16+00:00
1,1655968201422012418,2023-05-09 16:09:20+00:00,If Legacy Media is going to shove bigotry porn...,TheRabbitHole84,1409,412,5328,14775379.0,633,1655968899903418373,Massive public manipulation,9811,2694,49528,534,14404853.0,1241,2023-05-09 16:12:06+00:00
2,1647327385342320640,2023-04-15 19:53:49+00:00,Starlink provides internet to the most remote ...,teslaownersSV,1253,800,8472,20877634.0,173,1647339741610926080,💯,8257,4793,106036,376,18520248.0,366,2023-04-15 20:42:55+00:00
3,1646226917387796491,2023-04-12 19:00:57+00:00,National Public Radio Denies Being National Or...,TheBabylonBee,4781,701,37752,18478251.0,167,1646228474628280326,🤣🤣,10198,5076,108462,430,16690340.0,340,2023-04-12 19:07:08+00:00
4,1640016339011076097,2023-03-26 15:42:19+00:00,Arthur C. Clarke about the future of AI. \n—21...,Rainmaker1973,5409,737,20773,27514757.0,3333,1640171198091866114,Prescient,9193,5118,56272,572,25169601.0,3792,2023-03-27 01:57:41+00:00


In [6]:
quote_df.drop(['orig_tweet_id', 'orig_tweet_text', 'orig_tweet_username', 'orig_tweet_retweet_count', 'orig_tweet_reply_count', 'orig_tweet_like_count', 'orig_tweet_view_count', 'orig_tweet_bookmark_count'], axis=1, inplace=True)
quote_df.head()

,orig_tweet_created_at,musk_tweet_id,musk_quote_tweet,musk_quote_retweet_count,musk_quote_reply_count,musk_quote_like_count,musk_quote_quote_count,musk_quote_view_count,musk_quote_bookmark_count,musk_quote_created_at
0,2023-05-09 16:45:41+00:00,1655978502187778073,Yup,3255,3747,39533,225,11392206.0,281,2023-05-09 16:50:16+00:00
1,2023-05-09 16:09:20+00:00,1655968899903418373,Massive public manipulation,9811,2694,49528,534,14404853.0,1241,2023-05-09 16:12:06+00:00
2,2023-04-15 19:53:49+00:00,1647339741610926080,💯,8257,4793,106036,376,18520248.0,366,2023-04-15 20:42:55+00:00
3,2023-04-12 19:00:57+00:00,1646228474628280326,🤣🤣,10198,5076,108462,430,16690340.0,340,2023-04-12 19:07:08+00:00
4,2023-03-26 15:42:19+00:00,1640171198091866114,Prescient,9193,5118,56272,572,25169601.0,3792,2023-03-27 01:57:41+00:00


In [7]:
elon_tweets_df.drop(['musk_tweet_id', 'musk_quote_tweet', 'musk_quote_retweet_count', 'musk_quote_reply_count', 'musk_quote_like_count', 'musk_quote_quote_count', 'musk_quote_view_count', 'musk_quote_bookmark_count', 'musk_quote_created_at'], axis=1, inplace=True)
elon_tweets_df.head()

C:\Users\1alib\AppData\Local\Temp\ipykernel_17216\2747713613.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  elon_tweets_df.drop(['musk_tweet_id', 'musk_quote_tweet', 'musk_quote_retweet_count', 'musk_quote_reply_count', 'musk_quote_like_count', 'musk_quote_quote_count', 'musk_quote_view_count', 'musk_quote_bookmark_count', 'musk_quote_created_at'], axis=1, inplace=True)


,orig_tweet_id,orig_tweet_created_at,orig_tweet_text,orig_tweet_username,orig_tweet_retweet_count,orig_tweet_reply_count,orig_tweet_like_count,orig_tweet_view_count,orig_tweet_bookmark_count
14,1668843112955891712,2023-06-14 04:49:38+00:00,Bold,elonmusk,49375,18263,334997,127858792.0,6344
21,1665784889008873476,2023-06-05 18:17:20+00:00,https://t.co/OT6bqzUK8P,elonmusk,10205,7183,58206,34809220.0,2905
40,1656748197308674048,2023-05-11 19:48:45+00:00,Excited to announce that I’ve hired a new CEO ...,elonmusk,30150,31364,361878,120525183.0,2900
53,999367582271422464,2018-05-23 19:12:53+00:00,Going to create a site where the public can ra...,elonmusk,41664,17931,235900,NaN,1333
75,1619415871902056449,2023-01-28 19:23:25+00:00,"@atensnut Nonetheless, I apologize",elonmusk,414,1166,8046,1923932.0,59


In [8]:
quote_df.to_csv('quote_tweets.csv', index = False)

In [9]:
elon_tweets_df.to_csv('original_tweets.csv', index = False)